todo
1. optimize hyperparameters
2. clean dataset
    - balance
3. data augmentation
4. add requirments.txt
5. correct title (of file as well)
6. go through all variable names
7. ask which types of groups should "be in focus". Do disabled people count?
8. more advaned duplicate dropping?
9. balancing the dataset needed?
10. dedicated train test split?
11. data augmentation only on train?
12. remove duplicates a second time
13. train test split after tokenization?
14. change train test split ratio?
15. wtf is "output_dim"?

# Hate Detection

In [243]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix, f1_score

from keras.callbacks import EarlyStopping
from keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


from random import sample

### Data Exploration

In [244]:
data = pd.read_csv('hate-text.csv')
data

,case_id,test_case,label_gold,target_ident
0,1,I hate women.,hateful,women
1,2,I hate trans people.,hateful,trans people
2,3,I hate gay people.,hateful,gay people
3,4,I hate black people.,hateful,black people
4,5,I hate disabled people.,hateful,disabled people
...,...,...,...,...
3896,3897,Gay people are all complete 4ssholes.,hateful,gay people
3897,3898,Black people are all complete 4ssholes.,hateful,black people
3898,3899,Disabled people are all complete 4ssholes.,hateful,disabled people
3899,3900,Muslims are all complete 4ssholes.,hateful,Muslims


The dataset contains some missing values in the target_ident column, which we'll have to deal with later

In [245]:
# Using display() to display both at the same time. Won't comment any future use of display() (for obvious reasons)
display(
    data.isnull().any(),
    data[data.isna().any(axis=1)]
)

case_id         False
test_case       False
label_gold      False
target_ident     True
dtype: bool

,case_id,test_case,label_gold,target_ident
1271,1272,"Fucking hell, what a day.",non-hateful,NaN
1272,1273,That's a great fucking idea.,non-hateful,NaN
1273,1274,No fucking way.,non-hateful,NaN
1274,1275,I'm so fucking excited.,non-hateful,NaN
1275,1276,Are you fucking kidding me?,non-hateful,NaN
...,...,...,...,...
3097,3098,Pro lifers should get the fuck out of our coun...,non-hateful,NaN
3098,3099,Most gun dealers are seriously fucked in the h...,non-hateful,NaN
3099,3100,"By nature, corporate accountants are just shit...",non-hateful,NaN
3100,3101,Most real estate agents are absolute bastards.,non-hateful,NaN


No duplicate texts found

In [246]:
print(f'''duplicate texts: {data['test_case'].duplicated().any()}''')

duplicate texts: False


While the dataset is quite balanced in the target groups, it's still assymetric when it comes to the number of hate/non-hate texts

In [247]:
display(
    pd.DataFrame(data['label_gold'].value_counts()),
    pd.DataFrame(data['target_ident'].value_counts())
)

,count
label_gold,
hateful,2659
non-hateful,1242


,count
target_ident,
gay people,577
women,535
disabled people,510
Muslims,510
black people,504
trans people,485
immigrants,485


No unusually short/long texts

In [248]:
longest_text_length = data['test_case'].str.len().max()
shortest_text_length = data['test_case'].str.len().min()

longest_text_index = data['test_case'].str.len().idxmax()
shortest_text_index = data['test_case'].str.len().idxmin()

print(
    f'''Shortest: {longest_text_length}, "{data['test_case'][longest_text_index]}"\n'''
    f'''Longest: {shortest_text_length}, "{data['test_case'][shortest_text_index]}"'''
)

Shortest: 104, "When will it finally be clear to people that it's in a disabled person's nature to be an annoying prick?"
Longest: 11, "Ihate women"


### Data cleaning

Drop and rename columns to something more suitable

In [249]:
data = data.drop('case_id', axis=1)
data = data.rename(
    columns=
    {
        'test_case': 'text',
        'label_gold': 'label',
        'target_ident': 'target',
    }
)

data.head()

,text,label,target
0,I hate women.,hateful,women
1,I hate trans people.,hateful,trans people
2,I hate gay people.,hateful,gay people
3,I hate black people.,hateful,black people
4,I hate disabled people.,hateful,disabled people


It appears as though non of the missing target data have any relevant hate speech associated with them. This can easily be varified by manually looking at the dataset (given its tiny size). But alas, given my horrendous dyslexia i decided to programmatically varify it as a sanity check

In [250]:
target_search_words = [
    'gay',
    'women',
    'disabled',
    'Muslims',
    'black',
    'trans',
    'immigrants'
]

# Get all rows targets with NaNs
data_target_nans = data[data['target'].isna()]

for index, row in data_target_nans.iterrows():
    for word in target_search_words:
        # Split by words and search each one
        if word in row['text'].split(' '):
            print(f'''found target word in: "{row['text']}" row''')

I'm now confident in dropping all rows with target NaN values

In [251]:
data = data.dropna()

Balance the dataset

In [252]:
hateful = data['label'].value_counts()['hateful']
non_hateful = data['label'].value_counts()['non-hateful']

# Because we've got far more hateful texts
number_of_rows_to_drop = hateful - non_hateful

# randomly sample the rows and and drop them
rows_indexes_to_remove = data[data['label'] == 'hateful'].sample(n=number_of_rows_to_drop).index
data = data.drop(rows_indexes_to_remove)

display(
    pd.DataFrame(data['label'].value_counts()),
    pd.DataFrame(data['target'].value_counts())
)

,count
label,
hateful,947
non-hateful,947


,count
target,
gay people,317
women,292
Muslims,266
black people,265
disabled people,261
immigrants,248
trans people,245


### Data preprocessing

Shuffle the dataset

In [253]:
# Shuffles and returns the entire dataset as frac=1
data = data.sample(frac=1)

# Reset index after having both balanced and shuffled
data = data.reset_index(drop=True)
data

,text,label,target
0,The world would be better off without Muslims ...,hateful,Muslims
1,She should be kicked off the platform for sayi...,non-hateful,immigrants
2,Typical of an immigrant to be that stup1d.,hateful,immigrants
3,"She's got a fag in her mouth, and she's squint...",non-hateful,gay people
4,Immigrants are parasites to our society.,hateful,immigrants
...,...,...,...
1889,There is a woman on my football team.,non-hateful,women
1890,Immigrants are human garbage.,hateful,immigrants
1891,"If you say ""I hate women"", then you are a bigot!",non-hateful,women
1892,Women being in charge is a danger to our count...,hateful,women


In [254]:
print(
    f'''Shortest: {data['text'].str.len().min()}\n'''
    f'''Longest: {data['text'].str.len().max()}'''
)

Shortest: 14
Longest: 101


In [255]:
max_words_to_use = 1000

# Tokenize the text data (convert them into "sequences")
tokenizer = Tokenizer(num_words=max_words_to_use) # Consider only using the top 1000 words
tokenizer.fit_on_texts(data['text'])
tokenized_texts = tokenizer.texts_to_sequences(data['text'])

# printing using loop for easier viewing (won't mention again)
for i in range(5):
    print(f'seq {i}: {tokenized_texts[i]}')

seq 0: [10, 137, 40, 9, 223, 104, 143, 23, 7, 4]
seq 1: [224, 27, 9, 306, 104, 10, 307, 20, 61, 24, 27, 8, 9, 182]
seq 2: [50, 19, 92, 62, 3, 9, 11, 499]
seq 3: [559, 560, 5, 160, 12, 265, 728, 35, 559, 729, 265, 730, 424, 10, 731]
seq 4: [24, 2, 308, 3, 74, 75]


The longest series of tokenized words is only 20 items. Thus, we'll set our padding length accordingly.

In [256]:
longest_tokenized_text = max(tokenized_texts, key=len)
print(f'longest sequence of tokenized text: {len(longest_tokenized_text)}')

longest sequence of tokenized text: 20


In [257]:
max_sequence_length = 20

# Pad the sequences to make them of uniform length
tokenized_padded_texts = pad_sequences(tokenized_texts, maxlen=max_sequence_length)

for i in range(5):
    print(f'seq {i}: {tokenized_padded_texts[i]}')

seq 0: [  0   0   0   0   0   0   0   0   0   0  10 137  40   9 223 104 143  23
   7   4]
seq 1: [  0   0   0   0   0   0 224  27   9 306 104  10 307  20  61  24  27   8
   9 182]
seq 2: [  0   0   0   0   0   0   0   0   0   0   0   0  50  19  92  62   3   9
  11 499]
seq 3: [  0   0   0   0   0 559 560   5 160  12 265 728  35 559 729 265 730 424
  10 731]
seq 4: [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  24   2 308   3
  74  75]


In [258]:
# One-hot encode the labels
label_binarizer = LabelBinarizer()
onehot_encoded_labels = label_binarizer.fit_transform(data['label'])
onehot_encoded_targets = label_binarizer.fit_transform(data['target'])

print(f'labels: {onehot_encoded_labels[0]}')
print(f'targets: {onehot_encoded_targets[0]}\n')

for i in range(10):
    # the output looks identical for both
    print(f'label/target {i}: {onehot_encoded_labels[i][0]}')

labels: [0]
targets: [1 0 0 0 0 0 0]

label/target 0: 0
label/target 1: 1
label/target 2: 0
label/target 3: 1
label/target 4: 0
label/target 5: 1
label/target 6: 1
label/target 7: 1
label/target 8: 0
label/target 9: 0


In [259]:
train_texts, test_texts, train_labels, test_labels, train_targets, test_targets = train_test_split(
    tokenized_padded_texts,
    onehot_encoded_labels,
    onehot_encoded_targets,
    test_size=0.2
)

print(
    f'train_texts shape: {np.shape(train_texts)}\n'
    f'train_labels shape: {np.shape(train_labels)}\n'
    f'train_targets shape: {np.shape(train_targets)}\n'
    f'test_texts shape: {np.shape(test_texts)}\n'
    f'test_labels shape: {np.shape(test_labels)}\n'
    f'test_targets shape: {np.shape(test_targets)}'
)

train_texts shape: (1515, 20)
train_labels shape: (1515, 1)
train_targets shape: (1515, 7)
test_texts shape: (379, 20)
test_labels shape: (379, 1)
test_targets shape: (379, 7)


In [260]:
#for i in [train_labels]
model = Sequential([
    Embedding(
        input_dim=max_words_to_use,
        input_length=max_sequence_length,
        output_dim=32
    ),
    LSTM(units=32, activation='tanh'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Using a patience and start_from_epoch of 10 as the model is still all over the place before that
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, start_from_epoch=10)

# Will update the model objects inside the dictionary
history = model.fit(train_texts, train_labels, epochs=100, validation_split=0.2, callbacks=[early_stopping], verbose=1)

Epoch 1/100


c:\Users\Kevin\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\util\dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


38/38 [==============================] - 7s 42ms/step - loss: 0.0000e+00 - accuracy: 0.4827 - val_loss: 0.0000e+00 - val_accuracy: 0.5314
Epoch 2/100
38/38 [==============================] - 1s 20ms/step - loss: 0.0000e+00 - accuracy: 0.4851 - val_loss: 0.0000e+00 - val_accuracy: 0.5314
Epoch 3/100
38/38 [==============================] - 1s 20ms/step - loss: 0.0000e+00 - accuracy: 0.4851 - val_loss: 0.0000e+00 - val_accuracy: 0.5314
Epoch 4/100
38/38 [==============================] - 1s 19ms/step - loss: 0.0000e+00 - accuracy: 0.4851 - val_loss: 0.0000e+00 - val_accuracy: 0.5314
Epoch 5/100
38/38 [==============================] - 1s 18ms/step - loss: 0.0000e+00 - accuracy: 0.4851 - val_loss: 0.0000e+00 - val_accuracy: 0.5314
Epoch 6/100
38/38 [==============================] - 1s 20ms/step - loss: 0.0000e+00 - accuracy: 0.4851 - val_loss: 0.0000e+00 - val_accuracy: 0.5314
Epoch 7/100
38/38 [==============================] - 1s 21ms/step - loss: 0.0000e+00 - accuracy: 0.4851 - val_lo